In [2]:
from cv2 import AGAST_FEATURE_DETECTOR_AGAST_5_8
from src.getter import Get
import os
from models.sqliteModels import generate_symbols


In [3]:
symbol = generate_symbols()

In [9]:
stocks = [next(symbol) for i in range(100)]


# Main Code: 

In [15]:
from src.getter import Get, stock_chunks
from src.pricehistory import price_history
from src.fundamentals import fundamental
from src.quotes import quote
from sqlalchemy import create_engine
from models.sqliteModels import generate_symbols
from models.mysql_db import insert_quote_and_fundamental_data_mysql, connection_uri, marketdata_db
import pandas as pd
import os


In [3]:
# create a Get object
get = Get()

In [4]:
# =========================================================================== #
# Get the quote and fundamental data using stock_chunks
quote_data = pd.concat([quote.data(each) for each in stock_chunks])
fundamental_data = pd.concat([fundamental.data(each)
                              for each in stock_chunks])

In [10]:
# create the marketdata database engine:
engine = connection_uri + '/' + marketdata_db


In [11]:
# insert the quote and fundamental data
insert_quote_and_fundamental_data_mysql(
    quote_data, fundamental_data, marketdata_engine)


ValueError: [-] Data not inserted correctly. Make sure datatype is correct

In [ ]:
# Generator function that will generate one symbol at a time for price
# history data. This way the function will run once for each symbol
# stock = generate_symbols()
# =========================================================================== #

In [ ]:
# create the quote_df to pass to the already created functions
print(f"[1.3] Retrieve data from DataFrame's symbol column")
_stocks = companies_df["symbol"]

# #####################################################################
# CREATE COMPANIES LIST:
#####################################################################
# --->[6] Create Chunked list
stock_chunks = list(get.chunks(list(set(_stocks)), 200))

# Give quote and fundamental data an variable to save to:
# Get Quote Data
print("[3] Retrieve Quote and Fundamental data..")
quote_data = pd.concat([get.quotes(each) for each in stock_chunks])
print("[3.1] Quote Data Retrieved")

# Get Fundamental Data
fundamental_data = pd.concat([get.fundamentals(each)
                              for each in stock_chunks])
print("[3.2] Fundamental Data Retrieved")

print("[4] Insert Quote and Fundamental Data")
# Insert the fundamental and quote data to the database:
insert_quote_and_fundamental_data(quote_data=quote_data,
                                  fundamental_data=fundamental_data, db=db)

# print(quote_data)
